In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
folder = 'data_cleaned/'

collisions = pd.read_csv(folder+"collisions_cleaned.csv", dtype={'case_id': str}, low_memory=False)
parties = pd.read_csv(folder+"parties_cleaned.csv", dtype={'case_id': str}, low_memory=False)
victims = pd.read_csv(folder+"victims_cleaned.csv", dtype={'case_id': str}, low_memory=False)

In [3]:
tables_folder = 'tables/'

## COLLISION ENTITY

In [4]:
collisions.process_date = pd.to_datetime(collisions.process_date).dt.date

In [5]:
# create column datetime by merging collision_date and collision_time
collisions['datetime'] = pd.to_datetime(collisions.collision_date + ' ' + collisions.collision_time)
collisions.loc[collisions['datetime'].isnull(), "datetime"] = pd.to_datetime(collisions[collisions.collision_time.isnull()]["collision_date"])

In [6]:
cols_from_collision = ["case_id", "datetime", "officer_id", "type_of_collision", "process_date",
        "primary_collision_factor", "collision_severity", "tow_away", "hit_and_run"]
collision_entity = collisions[cols_from_collision]
collision_entity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678058 entries, 0 to 3678057
Data columns (total 9 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   case_id                   object        
 1   datetime                  datetime64[ns]
 2   officer_id                object        
 3   type_of_collision         object        
 4   process_date              object        
 5   primary_collision_factor  object        
 6   collision_severity        object        
 7   tow_away                  object        
 8   hit_and_run               object        
dtypes: datetime64[ns](1), object(8)
memory usage: 252.6+ MB


In [7]:
# add column school_bus_related from parties
bus_related_ids = parties[parties['school_bus_related']=="T"].case_id.unique()
collision_entity['school_bus_related'] = "F"
collision_entity.loc[collision_entity['case_id'].isin(bus_related_ids.tolist()), "school_bus_related"] = "T"

In [8]:
collision_entity.head()

,case_id,datetime,officer_id,type_of_collision,process_date,primary_collision_factor,collision_severity,tow_away,hit_and_run,school_bus_related
0,0000001,2002-01-18 15:30:00,16418,rear end,2002-06-14,vehicle code violation,property damage only,T,not hit and run,F
1,0000002,2002-02-13 19:30:00,14360,hit object,2002-03-29,vehicle code violation,property damage only,F,misdemeanor,F
2,0000003,2002-02-11 15:30:00,16117,hit object,2003-07-15,vehicle code violation,property damage only,F,misdemeanor,F
3,0000004,2002-02-12 07:45:00,6078,sideswipe,2002-05-22,vehicle code violation,property damage only,F,misdemeanor,F
4,0000005,2002-02-14 11:35:00,13851,overturned,2003-07-28,vehicle code violation,severe injury,T,not hit and run,F


In [9]:
collision_entity.datetime.isnull().sum()

0

In [10]:
def save_df(df, name):
    df.to_csv(tables_folder + name, index=False)

## VIOLATION ENTITY

In [11]:
def create_entity(df, cols, id_name):
    
    df_entity = df[cols].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': id_name})
    df_entity[id_name] = df_entity[id_name].apply(lambda x: x+1)
    
    return df_entity

In [12]:
cols_from_collision = ["pcf_violation", "pcf_violation_category", "pcf_violation_subsection"]

violation_entity = create_entity(collisions, cols_from_collision, 'vid')

In [13]:
violation_entity.head()

,vid,pcf_violation,pcf_violation_category,pcf_violation_subsection
0,1,22107.0,improper turning,NaN
1,2,22515.0,hazardous parking,A
2,3,23114.0,other hazardous violation,A
3,4,22450.0,traffic signals and signs,A
4,5,22350.0,speeding,NaN


In [14]:
# Save violation_entity
save_df(violation_entity, "violation_entity.csv")

## VIOLATED RELATIONSHIP (ADD VID TO COLLISIONS)

In [15]:
map_caseid_to_vid = pd.merge(collisions[['case_id'] + cols_from_collision], violation_entity, how='inner', on=cols_from_collision )[['case_id', 'vid']]
collision_entity = pd.merge(collision_entity, map_caseid_to_vid, how='inner', on=['case_id'])
collision_entity.head()

,case_id,datetime,officer_id,type_of_collision,process_date,primary_collision_factor,collision_severity,tow_away,hit_and_run,school_bus_related,vid
0,0000001,2002-01-18 15:30:00,16418,rear end,2002-06-14,vehicle code violation,property damage only,T,not hit and run,F,1
1,0000002,2002-02-13 19:30:00,14360,hit object,2002-03-29,vehicle code violation,property damage only,F,misdemeanor,F,2
2,0000003,2002-02-11 15:30:00,16117,hit object,2003-07-15,vehicle code violation,property damage only,F,misdemeanor,F,3
3,0000004,2002-02-12 07:45:00,6078,sideswipe,2002-05-22,vehicle code violation,property damage only,F,misdemeanor,F,4
4,0000005,2002-02-14 11:35:00,13851,overturned,2003-07-28,vehicle code violation,severe injury,T,not hit and run,F,5


In [16]:
collision_entity.shape[0] == collisions.shape[0]

True

## LOCATION ENTITY

In [17]:
cols_from_collision = ['jurisdiction', 'location_type', 'ramp_intersection', 'county_city_location', 'population']

location_entity = create_entity(collisions, cols_from_collision, 'lid')

In [18]:
location_entity.head()

,lid,jurisdiction,location_type,ramp_intersection,county_city_location,population
0,1,9575.0,NaN,NaN,1900,9.0
1,2,9420.0,NaN,NaN,1500,9.0
2,3,9420.0,highway,NaN,1502,6.0
3,4,9420.0,NaN,NaN,1502,6.0
4,5,9420.0,highway,NaN,1500,9.0


In [19]:
# Save location_entity
save_df(location_entity, "location_entity.csv")

## LOCATED RELATIONSHIP (ADD LID TO COLLISIONS)

In [20]:
map_caseid_to_lid = pd.merge(collisions[['case_id'] + cols_from_collision], location_entity, how='inner', on=cols_from_collision )[['case_id', 'lid']]
collision_entity = pd.merge(collision_entity, map_caseid_to_lid, how='inner', on=['case_id'])
collision_entity.head()

,case_id,datetime,officer_id,type_of_collision,process_date,primary_collision_factor,collision_severity,tow_away,hit_and_run,school_bus_related,vid,lid
0,0000001,2002-01-18 15:30:00,16418,rear end,2002-06-14,vehicle code violation,property damage only,T,not hit and run,F,1,1
1,0000002,2002-02-13 19:30:00,14360,hit object,2002-03-29,vehicle code violation,property damage only,F,misdemeanor,F,2,2
2,0000003,2002-02-11 15:30:00,16117,hit object,2003-07-15,vehicle code violation,property damage only,F,misdemeanor,F,3,3
3,0000004,2002-02-12 07:45:00,6078,sideswipe,2002-05-22,vehicle code violation,property damage only,F,misdemeanor,F,4,2
4,0000005,2002-02-14 11:35:00,13851,overturned,2003-07-28,vehicle code violation,severe injury,T,not hit and run,F,5,2


In [21]:
collision_entity.shape[0] == collisions.shape[0]

True

## CONDITIONS ENTITY

In [22]:
cols_from_collision = ['lighting', 'road_surface']

conditions_entity = create_entity(collisions, cols_from_collision, 'cid')

In [23]:
conditions_entity.head()

,cid,lighting,road_surface
0,1,daylight,dry
1,2,NaN,dry
2,3,dusk or dawn,dry
3,4,dark with street lights,dry
4,5,dark with no street lights,dry


In [24]:
save_df(conditions_entity, "conditions_entity.csv")

## CONDITIONS RELATIONSHIP (ADD CID TO COLLISIONS)

In [25]:
map_caseid_to_cid = pd.merge(collisions[['case_id'] + cols_from_collision], conditions_entity, how='inner', on=cols_from_collision )[['case_id', 'cid']]
collision_entity = pd.merge(collision_entity, map_caseid_to_cid, how='inner', on=['case_id'])
collision_entity.head()

,case_id,datetime,officer_id,type_of_collision,process_date,primary_collision_factor,collision_severity,tow_away,hit_and_run,school_bus_related,vid,lid,cid
0,0000001,2002-01-18 15:30:00,16418,rear end,2002-06-14,vehicle code violation,property damage only,T,not hit and run,F,1,1,1
1,0000002,2002-02-13 19:30:00,14360,hit object,2002-03-29,vehicle code violation,property damage only,F,misdemeanor,F,2,2,2
2,0000003,2002-02-11 15:30:00,16117,hit object,2003-07-15,vehicle code violation,property damage only,F,misdemeanor,F,3,3,1
3,0000004,2002-02-12 07:45:00,6078,sideswipe,2002-05-22,vehicle code violation,property damage only,F,misdemeanor,F,4,2,1
4,0000005,2002-02-14 11:35:00,13851,overturned,2003-07-28,vehicle code violation,severe injury,T,not hit and run,F,5,2,1


In [26]:
collision_entity.shape[0] == collisions.shape[0]

True

In [27]:
# Save collision_entity
save_df(collision_entity, "collision_entity.csv")

## WEATHER ENTITY

In [28]:
def normal_form(df, attribute, id_name):
    df_normal = pd.concat([df[~df[attribute + '_1'].isna()][[id_name, attribute + '_1']]\
                           .rename(columns={attribute + '_1' : attribute}),
                            df[~df[attribute + '_2'].isna()][[id_name, attribute + '_2']]\
                           .rename(columns={attribute + '_2' : attribute})])
    return df_normal

In [29]:
weather_entity = normal_form(collisions, "weather", "case_id")

In [30]:
weather_entity.head()

,case_id,weather
0,0000001,clear
1,0000002,clear
2,0000003,clear
3,0000004,clear
4,0000005,clear


In [31]:
save_df(weather_entity, "weather_entity.csv")

## ROAD CONDITIONS ENTITY

In [32]:
roadcondition_entity = normal_form(collisions, "road_condition", "case_id")

In [33]:
roadcondition_entity.head()

,case_id,road_condition
0,0000001,normal
1,0000002,normal
2,0000003,normal
3,0000004,normal
4,0000005,normal


In [34]:
save_df(roadcondition_entity, "roadcondition_entity.csv")

## PARTY ENTITY

In [35]:
cols_from_parties = ['id', 'case_id', 'party_number', 'party_type', 'party_age', 'party_sex',
                   'party_sobriety', 'at_fault', 'party_drug_physical', 'cellphone_use', 'movement_preceding_collision',
                   'hazardous_materials', 'financial_responsibility']

party_entity = parties[cols_from_parties].rename(columns={'id': 'pid'})

In [36]:
party_entity.head()

,pid,case_id,party_number,party_type,party_age,party_sex,party_sobriety,at_fault,party_drug_physical,cellphone_use,movement_preceding_collision,hazardous_materials,financial_responsibility
0,8,0000001,1,driver,84.0,F,A,T,NaN,C,other,F,Y
1,9,0000001,2,parked vehicle,NaN,NaN,H,F,H,D,parked,F,O
2,10,0000002,1,other,NaN,NaN,H,F,H,NaN,proceeding straight,F,O
3,11,0000003,1,driver,20.0,F,A,F,NaN,D,proceeding straight,F,Y
4,12,0000003,2,driver,NaN,NaN,G,T,G,D,proceeding straight,F,NaN


In [37]:
save_df(party_entity, "party_entity.csv")

## PARTY EQUIPMENT

In [38]:
partyEquipment = normal_form(parties, "party_safety_equipment", "id").rename(columns={'id': 'pid', "party_safety_equipment": "equipment"})

In [39]:
partyEquipment.head()

,pid,equipment
0,8,Lap/Shoulder Harness Used
3,11,Lap/Shoulder Harness Used
6,14,Lap/Shoulder Harness Used
7,15,Lap/Shoulder Harness Used
8,16,Lap/Shoulder Harness Used


In [40]:
save_df(partyEquipment, "partyEquipment.csv")

## VEHICLE ENTITY

In [41]:
cols_from_parties = ['id', 'statewide_vehicle_type', 'vehicle_make', 'vehicle_year']

vehicle_entity = parties[cols_from_parties].rename(columns={'id': 'pid'})

In [42]:
vehicle_entity.head()

,pid,statewide_vehicle_type,vehicle_make,vehicle_year
0,8,passenger car,FORD,2000.0
1,9,passenger car,BUICK,1992.0
2,10,pickup or panel truck,TOYOTA,NaN
3,11,passenger car,FORD,1995.0
4,12,pickup or panel truck,NaN,NaN


In [43]:
save_df(vehicle_entity, "vehicle_entity.csv")

## OTHER ASSOCIATED FACTOR ENTITY

In [44]:
otherAssociatedFactor_entity = normal_form(parties, "other_associate_factor", "id").rename(columns={'id': 'pid', "other_associate_factor": "factor"})

In [45]:
otherAssociatedFactor_entity.head()

,pid,factor
0,8,Uninvolved Vehicle
2,10,Runaway Vehicle
5,13,Violation
13,21,Stop and Go Traffic
30,38,Violation


In [46]:
save_df(otherAssociatedFactor_entity, 'otherAssociatedFactor_entity.csv')

## VICTIM ENTITY

In [47]:
victims.columns

Index(['case_id', 'id', 'party_number', 'victim_age',
       'victim_degree_of_injury', 'victim_ejected', 'victim_role',
       'victim_safety_equipment_1', 'victim_safety_equipment_2',
       'victim_seating_position', 'victim_sex',
       'victim_seating_position_code'],
      dtype='object')

In [48]:
cols = ['vid', 'pid', 'victim_role', 'victim_age', 'victim_degree_of_injury', 'victim_ejected', 'victim_sex',
                    'victim_seating_position', 'victim_seating_position_code']

victim_entity = pd.merge(victims.rename(columns={'id': 'vid'}), parties[['case_id', 'party_number', 'id']].rename(columns={'id': 'pid'}), how='inner', on=['case_id', 'party_number'])[cols]

In [49]:
victim_entity.head()

vid  pid victim_role  victim_age victim_ejected victim_sex  \
0    3   11   Passenger        21.0    Not Ejected          M   
1    4   15      Driver        44.0    Not Ejected          M   
2    5   20   Passenger        59.0    Not Ejected          F   
3    6   21   Passenger        31.0    Not Ejected          M   
4    7   23   Passenger        14.0    Not Ejected          F   

  victim_seating_position  victim_seating_position_code  
0               Passenger                           3.0  
1                  Driver                           1.0  
2               Passenger                           3.0  
3               Passenger                           3.0  
4               Passenger                           6.0

In [50]:
victim_entity.shape[0] == victims.shape[0]

True

In [51]:
save_df(victim_entity, "victim_entity.csv")

## VICTIM EQUIPMENT

In [52]:
victimEquipment = normal_form(victims, "victim_safety_equipment", "id").rename(columns={'id': 'vid', "victim_safety_equipment": "equipment"})

In [53]:
victimEquipment.head()

,vid,equipment
0,3,Lap/Shoulder Harness Used
1,4,Lap/Shoulder Harness Used
2,5,Lap/Shoulder Harness Used
3,6,Lap/Shoulder Harness Used
4,7,Lap Belt Used


In [54]:
save_df(victimEquipment, "victimEquipment.csv")

In [55]:
collision_entity.isna().mean()

case_id                     0.000000e+00
datetime                    0.000000e+00
officer_id                  2.504583e-03
type_of_collision           7.178788e-03
process_date                0.000000e+00
primary_collision_factor    8.558593e-03
collision_severity          0.000000e+00
tow_away                    3.882212e-03
hit_and_run                 2.718826e-07
school_bus_related          0.000000e+00
vid                         0.000000e+00
lid                         0.000000e+00
cid                         0.000000e+00
dtype: float64